In [2]:
import numpy as np
import numpy.ma as ma
import math
import datetime

lambdaa = 8
lambdas = 20
taus = 200
taua = 200

def unitTest():
    size = 10 
    myArray = np.zeros((size, size))
    for i in range (0, size):
        for j in range(i+1, i+3):
            if(i<size and j<size):
                myArray[i, j] = i * j
    print myArray
    np.savetxt('unitTest.txt', myArray)
    arrayPath(1, 1)
    

def frameDevFunction(i, j, v):
    frameCostValue = min(math.pow(((j-i)-v),2), taus)
    return frameCostValue

def frameAccelCost(h, i, j):
    frameaccelerationcost = min(math.pow(((j-i) - (h-i)),2), taua)
    return frameaccelerationcost

def arrayPath(extraDiag, v):
    
    #v is targetted speed up; v<=extraDiag
    print "Loading Array"
    errors = np.loadtxt('unitTest.txt')
    print "Array Loaded"
    #Can we remove the frameCount and just define it as the shape? Same with extra diag?
    frameCount = errors.shape[0]
    # extraDiag = 
    
    #Add our speed up cost function
    for i in range(0, frameCount):
        for j in range(i+1, i+extraDiag+2):
            if(j<frameCount and i<frameCount):
                errors[i,j] += lambdas * frameDevFunction(i, j, v)
    print "Speed Up Cost Added"
    print errors
    
    #Array is populated now assess costs
    costmatrix = np.zeros((frameCount, frameCount))
    #First Pass of Populate
    for i in range(1,frameCount):
        for j in range(i+1, i+extraDiag+2):
            n1 = []    
            #Populate a new list based on the for loop
            for k in range(i-1, i-(extraDiag+2),-1):
                #Populate our list
                n1.insert(k, (errors[k, i] + lambdaa*frameAccelCost(k, i, j)))
            if(i< frameCount and j<frameCount):
                errors[i, j]=np.min(n1)               
                #Is this next line correct or should we not build on the prior result? Pseudo-code is ambiguous          
                costmatrix[i, j] = i - ((extraDiag+1) - np.argmin(n1))   
    print "Cost Matrix Calculated"
                                             
    #Second Pass Trace Back Min Cost
    #How many frames back are we willing to go for our final frame?
    endrange = 3
    #Create a masked array that masks zeros in the last few rows and columns of our costmatrix
    maskedarray = ma.masked_equal(costmatrix[(frameCount-(endrange+1)):, (frameCount-(endrange+1)):], 0)
    #The argmin from this new masked array are then used for the s and d values, unravel and the addition
    #Of the frame count puts them into the correct values with regards to the size of our original cost matrix
    s= (np.unravel_index(maskedarray.argmin(), maskedarray.shape)[0]) + (frameCount-(endrange+1))
    d= (np.unravel_index(maskedarray.argmin(), maskedarray.shape)[1]) + (frameCount-(endrange+1))
    p = []
    while(s>1):
        #Prepend to List and work our way back up through the cost matrix
        p.insert(0, s)
        b = costmatrix[s, d]
        d = s
        s = b
    np.savetxt('framelist.txt',p)
    print p
    print "Frame Path Saved"

#arrayPath(15, 10)
unitTest()


[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   2.   3.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   6.   8.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  12.  15.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  20.  24.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  30.  35.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.  42.  48.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  56.  63.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.  72.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Loading Array
Array Loaded
Speed Up Cost Added
[[  0.   0.  20.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   2.  23.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   6.  28.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  12.  35.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  20.  44.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  30.  55.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.  42.  68.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  56.  83.

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:78: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
